<a href="https://colab.research.google.com/github/MudasirIrshad/Agentic_AI_Panaversity_Course/blob/main/Function_Calling_Tool_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -qU "google-generativeai"

In [5]:
from google.colab import userdata
import google.generativeai as genai

import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [7]:
genai.configure()

In [8]:
def enable_lights():
  """Turn on the lighting system"""
  print("LIGHTBOT: Lights enabled.")

def set_light_color(rgb_hex: str):
  """Set the light color, lights must be enabled for this to work."""
  print(f"LIGHTBOT: Lights set to {rgb_hex}.")

def disable_lights():
  """Turn off the lighting system"""
  print("LIGHTBOT: Lights disabled.")

In [49]:
light_controls = [enable_lights, set_light_color,disable_lights]
instruction = """you are a helpful lightning system bot. You can turn lights on
and off and you can set color of lights. Do not perform any other task"""

model = genai.GenerativeModel(
    "gemini-2.5-flash", tools= light_controls, system_instruction=instruction
)

In [50]:
chat = model.start_chat()

response = chat.send_message("set light color to red")
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "set_light_color",
                  "args": {
                    "rgb_hex": "#FF0000"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 141,
        "candidates_token_count": 23,
        "total_token_count": 281
      },
      "model_version": "gemini-2.5-flash"
    }),
)


#Practicing using Dictionary

In [112]:
students_details={
    "111": {
        "name": "Mudasir Irshad",
        "age": 20,
        "grade": "A"
    },
    "112": {
        "name": "Haris",
        "age": 22,
        "grade": "B"
    },
    "113": {
        "name": "Shoaib",
        "age": 21,
        "grade": "C"
    },
    "114": {
        "name": "Ajlal",
        "age": 23,
        "grade": "A"
    }
}

students_marks={
    "111":{
        "english": 50,
        "math": 60,
        "science": 70
    },
    "112":{
        "english": 60,
        "math": 70,
        "science": 80
    },
    "113":{
        "english": 70,
        "math": 80,
        "science": 90
    },
    "114":{
        "english": 80,
        "math": 90,
        "science": 100
    }
}

print(students_marks.get("112"))

{'english': 60, 'math': 70, 'science': 80}


In [114]:
def get_student_details(student_id:str):
  """Get student detail from students_details dictionary"""
  print("SCHOOLBOT: Student details retrieved.")
  return students_details.get(student_id)

def get_student_marks(student_id: str):
  """Get student marks from students_marks dictionary"""
  print("SCHOOLBOT: Student marks retrieved.")
  return students_marks.get(student_id)

def get_student_marks_and_details(student_id: str):
  """Get student marks and details from students_details and students_marks dictionary"""
  print("SCHOOLBOT: Student marks and details retrieved.")
  return students_details.get(student_id), students_marks.get(student_id)

instruction = """
You are a helpful school's student's details retrieval bot.
You can get specific student details by checking student id.
you can get specific student marks by checking student id.
you can get specific student marks and details by checking student id.
Do not perform any other tasks.
"""


model = genai.GenerativeModel(
    "gemini-2.0-flash-001", tools=[get_student_details,get_student_marks,get_student_marks_and_details], system_instruction=instruction
)



In [115]:
chat = model.start_chat()

response = chat.send_message("get student details and marks of student id 111")

function_name = response.candidates[0].content.parts[0].function_call.name
student_id = response.candidates[0].content.parts[0].function_call.args["student_id"]

if function_name == "get_student_details":
  print(get_student_details(student_id))
elif function_name == "get_student_marks":
  print(get_student_marks(student_id))
elif function_name == "get_student_marks_and_details":
  print(get_student_marks_and_details(student_id))

SCHOOLBOT: Student marks and details retrieved.
({'name': 'Mudasir Irshad', 'age': 20, 'grade': 'A'}, {'english': 50, 'math': 60, 'science': 70})
